# Example notebook for collecting data, training and evaluating the model

## Importing

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPRegressor
import xgboost as xgb

from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.metrics import root_mean_squared_error as rmse

import salishsea_tools.viz_tools as sa_vi

## Files Reading

In [ ]:
# The location of the file
ds = xr.open_dataset('/data/ibougoudis/MOAD/files/integrated_original.nc')
ds2 = xr.open_dataset('/data/ibougoudis/MOAD/files/external_inputs.nc')

ds = ds.isel(time_counter = (np.arange(0, len(ds.time_counter),2)), 
    y=(np.arange(ds.y[0], ds.y[-1], 5)), 
    x=(np.arange(ds.x[0], ds.x[-1], 5)))

ds2 = ds2.isel(time_counter = (np.arange(0, len(ds2.time_counter),2)), 
    y=(np.arange(ds2.y[0], ds2.y[-1], 5)), 
    x=(np.arange(ds2.x[0], ds2.x[-1], 5)))

# Selecting the first 2 years
dataset = ds.sel(time_counter = slice('2007-2-15', '2009-4-30'))
dataset2 = ds2.sel(time_counter = slice('2007-2-15', '2009-4-30'))


## Datasets Preparation

### Correlation Coefficients


| Diatom Biomass | Training | 2021 (annual) |2021 (mean daily) | 2021 (mean 5% low)| 2022 (annual) | 2022 (mean daily) | 2022 (mean 5% low) | 2023 (annual) | 2023 (mean daily) | 2023 (mean 5% low) | 2024 (annual) | 2024 (mean daily) | 2024 (mean 5% low)|
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| hist | .9 | .8 | .843 | | .7 | 0.8
| hist + xy | .9 | .8 | .843 | | .7 | 0.8
| hist + nitr | .9 | .8 | .843 | | .7 | 0.8
| hist + xy + nitr | .9 | .8 | .843 | | .7 | 0.8
| hist + fun_cl | .9 | .8 | .843 | | .7 | 0.8
| hist + fun_cl + nitr | .9 | .8 | .843 | | .7 | 0.8
| func_reg | .9 | .8 | .843 | | .7 | 0.8